In [1]:
import os
import pickle
import numpy as np
import datetime
import json
from multiprocessing import Process

from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications import VGG16, MobileNetV2

from models import *
from utils import regression_stats
from img_utils import data_to_df, preprocess_images, set_gpu, set_cpu

2024-04-22 10:50:42.753557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 10:50:42.753621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 10:50:42.754985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 10:50:42.763263: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 10:50:44.331866: W tensorflow/compiler/tf2

In [2]:
IMAGE_WIDTH: int = 448
IMAGE_HEIGHT: int = 448
MODELS_PATH: str = "./vision_models"
USE_GPU: bool = True

In [3]:
# Load Data
train_1_path: str = "../nybolig-scrape/output/train/train_1"
train_2_path: str = "../nybolig-scrape/output/train/train_2"
valid_path: str = "../nybolig-scrape/output/valid"
test_path: str = "../nybolig-scrape/output/test"

train1_df, train2_df, valid_df, test_df = data_to_df(
    [train_1_path, train_2_path, valid_path, test_path], preprocess=True
)

# train_images: np.array = preprocess_images(
#     train1_df["image_floorplan"], IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
# )
train_images: np.array = preprocess_images(
    train1_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
train_prices: np.array = train1_df["price"].values

valid_images: np.array = preprocess_images(
    valid_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
valid_prices: np.array = valid_df["price"].values

test_images: np.array = preprocess_images(
    test_df, "image_floorplan", IMAGE_WIDTH, IMAGE_HEIGHT, True, False, False
)
test_prices: np.array = test_df["price"].values

Processing ../nybolig-scrape/output/train/train_1: 100%|██████████| 311/311 [00:00<00:00, 1964500.82it/s]
Processing ../nybolig-scrape/output/train/train_2: 100%|██████████| 312/312 [00:00<00:00, 1561602.44it/s]
Preprocessing: 100%|██████████| 4/4 [00:00<00:00, 182.24it/s]


In [6]:
def save_model_and_evaluate(
    model, fit_history, test_images, test_prices, model_dir
):
    # Save Model
    print("Saving Model...")
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model.save(f"{model_dir}/model")

    # Save Training History
    with open(f"{model_dir}/history", "wb") as file_pi:
        pickle.dump(fit_history.history, file_pi)

    # Evaluate Model
    print("Evaluating Model...")
    test_predictions = model.predict(test_images)
    r2, mae, percentage_error, mse = regression_stats(test_prices, test_predictions)

    try:
        feature_importance = model.feature_importance_
    except AttributeError:
        feature_importance = None

    # Load existing evaluation data
    evaluation_file_path = f"{model_dir}/evaluation.json"
    evaluation_data = {}
    if os.path.exists(evaluation_file_path):
        with open(evaluation_file_path, "r") as json_file:
            evaluation_data = json.load(json_file)

    # Add new evaluation data
    new_evaluation = {
        "Timestamp": str(datetime.datetime.now()),
        "R2": r2,
        "MAE": mae,
        "Percentage Error": percentage_error,
        "MSE": mse,
        "Feature Importances": str(feature_importance) if feature_importance else None,
    }
    evaluation_data[len(evaluation_data)] = new_evaluation

    # Save updated evaluation data
    with open(evaluation_file_path, "w") as json_file:
        json.dump(evaluation_data, json_file, indent=4)

    # Compute median evaluation values from all instances
    r2_values = [evaluation_data[key]["R2"] for key in evaluation_data]
    mae_values = [evaluation_data[key]["MAE"] for key in evaluation_data]
    percentage_error_values = [
        evaluation_data[key]["Percentage Error"] for key in evaluation_data
    ]
    mse_values = [evaluation_data[key]["MSE"] for key in evaluation_data]

    median_evaluation_data = {
        "R2": np.median(r2_values),
        "MAE": np.median(mae_values),
        "Percentage Error": np.median(percentage_error_values),
        "MSE": np.median(mse_values),
    }

    with open(f"{model_dir}/median_evaluation.json", "w") as json_file:
        json.dump(median_evaluation_data, json_file, indent=4)

    print("Model Evaluation:")
    print(new_evaluation)
    print("Median Evaluation:")
    print(median_evaluation_data)

    print("Done!")


def train_save_model(
    model_func: object, args: tuple, test_images: np.array, test_prices: np.array, model_dir: str
):
    if USE_GPU:
        set_gpu()
    else:
        set_cpu()

    model, fit_history = model_func(*args)
    save_model_and_evaluate(model, fit_history, test_images, test_prices, model_dir)


def train_save_models(
    model_func: object, args: tuple, test_images: np.array, test_prices: np.array, model_dir: str
):
    if USE_GPU:
        set_gpu()
    else:
        set_cpu()

    models, fit_histories = model_func(*args)
    for model_idx, (model, fit_history) in enumerate(zip(models, fit_histories)):
        save_model_and_evaluate(
            model,
            fit_history,
            test_images,
            test_prices,
            f"{model_dir}_{model_idx}"
        )

In [7]:
MODEL_NAME: str = "VGG16"
FUNCTION: object = CNN_model
VARIABLES: tuple = (
    VGG16,
    True,
    train_images,
    train_prices,
    valid_images,
    valid_prices,
)
p = Process(
    target=train_save_model,
    args=(FUNCTION, VARIABLES, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
)

# MODEL_NAME: str = "N_CNN_MobileNetV2"
# FUNCTION: object = N_CNN_model
# VARIABLES: tuple = (
#     MobileNetV2,
#     train_images,
#     train_prices,
#     valid_images,
#     valid_prices,
#     3,
# )
# p = Process(
#     target=train_save_models,
#     args=(FUNCTION, VARIABLES, test_images, test_prices, f"{MODELS_PATH}/{MODEL_NAME}"),
# )
p.start()
p.join()

2024-04-22 10:54:36.449036: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.498913: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.499048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.503280: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.503518: I external/local_xla/xla/stream_executor

1 Physical GPUs, 1 Logical GPU


2024-04-22 10:54:36.685594: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.685904: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.685937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2024-04-22 10:54:36.686042: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 10:54:36.686077: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

Epoch 1/150


2024-04-22 10:54:44.648400: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-22 10:54:44.726745: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-22 10:54:44.924944: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-22 10:54:48.341362: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f51e1b70dd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-22 10:54:48.341433: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-04-22 10:54:48.348428: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713776088.420974   46590 device_compiler.

10/10 [==============================] - 16s 761ms/step - loss: 4097187.5000 - mean_absolute_error: 4095483.2500 - val_loss: 4385090.5000 - val_mean_absolute_error: 4383242.5000
Epoch 2/150
10/10 [==============================] - 3s 276ms/step - loss: 4024998.5000 - mean_absolute_error: 4022337.7500 - val_loss: 4167484.5000 - val_mean_absolute_error: 4163149.0000
Epoch 3/150
10/10 [==============================] - 3s 260ms/step - loss: 3518967.0000 - mean_absolute_error: 3512964.2500 - val_loss: 2969980.7500 - val_mean_absolute_error: 2961101.2500
Epoch 4/150
10/10 [==============================] - 3s 267ms/step - loss: 2082090.1250 - mean_absolute_error: 2071092.0000 - val_loss: 1918351.6250 - val_mean_absolute_error: 1905259.3750
Epoch 5/150
10/10 [==============================] - 3s 258ms/step - loss: 1896946.6250 - mean_absolute_error: 1884243.5000 - val_loss: 1908325.0000 - val_mean_absolute_error: 1896143.6250
Epoch 6/150
10/10 [==============================] - 3s 263ms/step

INFO:tensorflow:Assets written to: ./vision_models/MobileNetV2/model/assets


INFO:tensorflow:Assets written to: ./vision_models/MobileNetV2/model/assets


Evaluating Model...
6/6 [==============================] - 3s 381ms/step
Model Evaluation:
{'Timestamp': '2024-04-22 10:56:42.938143', 'R2': 0.3128360843724488, 'MAE': 1497950.018404908, 'Percentage Error': 55.91313852380283, 'MSE': 4457597925288.614, 'Feature Importances': None}
Median Evaluation:
{'R2': 0.3099037387253228, 'MAE': 1487103.299846626, 'Percentage Error': 55.67302026742497, 'MSE': 4476619904725.53}
Done!
